In [9]:
import numpy as np # library to handle data in a vectorized manner
!pip3 install --upgrade numpy

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests 
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

    100% |████████████████████████████████| 13.9MB 613kB/s eta 0:00:01████████████████       | 10.9MB 4.6MB/s eta 0:00:01
  Found existing installation: numpy 1.16.0
    Uninstalling numpy-1.16.0:
      Successfully uninstalled numpy-1.16.0
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### I find the coordinates of the first possible location

In [29]:
address = 'Piazzale Giulio Cesare, Milano, Italia'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x1 = location.latitude
longitude_y1 = location.longitude
print('The geographical coordinates of Piazzale Giulio Cesare are {}, {}.'.format(latitude_x1,longitude_y1))


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geographical coordinates of Piazzale Giulio Cesare are 45.4745032, 9.15559394876567.


### I create the map of Location_1

In [11]:
map_location_1 = folium.Map(location=[latitude_x1,longitude_y1], zoom_start=20)
map_location_1

### I set the Forusquare connection

In [12]:
CLIENT_ID = 'HMN0ROLECZHKTAGFQS4KCTMWDWX5FPCWEMWM5AABQNRAV5LN' 
CLIENT_SECRET = 'BYA5TUW2DOC02Z5NZZ5RLUPOAGWGKFQCLL4CFWMEI2VZIKUB'
VERSION = '20180609' 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HMN0ROLECZHKTAGFQS4KCTMWDWX5FPCWEMWM5AABQNRAV5LN
CLIENT_SECRET:BYA5TUW2DOC02Z5NZZ5RLUPOAGWGKFQCLL4CFWMEI2VZIKUB


In [13]:
lat_1 = latitude_x1
lon_1 = longitude_y1
nei_name = 'Location_1'
print('Latitude and longitude of {} are {} and {}'.format(nei_name, nei_lat, nei_lon))

Latitude and longitude of Location_1 are 45.4745032 and 9.15559394876567


### I look for the 100 venues that are  in a radius of 500 meters of Location 1

In [35]:
LIMIT = 100

In [36]:
radius = 500

In [16]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat_1, 
    lon_1, 
    radius, 
    LIMIT)

url



'https://api.foursquare.com/v2/venues/explore?&client_id=HMN0ROLECZHKTAGFQS4KCTMWDWX5FPCWEMWM5AABQNRAV5LN&client_secret=BYA5TUW2DOC02Z5NZZ5RLUPOAGWGKFQCLL4CFWMEI2VZIKUB&v=20180609&ll=45.4745032,9.15559394876567&radius=500&limit=100'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c542077dd5797192b3dfee5'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Zona 8',
  'headerFullLocation': 'Zona 8, Milan',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 45.4790032045, 'lng': 9.161999299786233},
   'sw': {'lat': 45.4700031955, 'lng': 9.149188597745107}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '538b48d0498ed1082716cafa',
       'name': 'Parco di CityLife',
       'location': {'address': 'Via Demetrio Stratos',
        'lat': 45.476115781018784,
        'lng': 9.156122142405946,
        'labeledLatLngs': [{'label': 'display',
          'lat': 45.476115781018784,


In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]


nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)


nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)


,name,categories,lat,lng
0,Parco di CityLife,Park,45.476116,9.156122
1,Il Viaggiator Goloso,Supermarket,45.473401,9.158814
2,Piazzale Giulio Cesare,Plaza,45.474118,9.155429
3,MI Cucina di Confine,Asian Restaurant,45.475338,9.159329
4,Piazza Tre Torri,Plaza,45.478005,9.156823
5,Bomaki,Sushi Restaurant,45.478223,9.154725
6,California Bakery,Bakery,45.477951,9.156397
7,Oren,Asian Restaurant,45.474612,9.151339
8,Venchi,Chocolate Shop,45.477766,9.156335
9,Hotel Campion,Hotel Bar,45.474777,9.151247


In [17]:
nearby_venues.shape

(42, 4)

In [21]:
Location_1_venues = nearby_venues
Location_1_venues

,name,categories,lat,lng
0,Parco di CityLife,Park,45.476116,9.156122
1,Il Viaggiator Goloso,Supermarket,45.473401,9.158814
2,Piazzale Giulio Cesare,Plaza,45.474118,9.155429
3,MI Cucina di Confine,Asian Restaurant,45.475338,9.159329
4,Piazza Tre Torri,Plaza,45.478005,9.156823
5,Bomaki,Sushi Restaurant,45.478223,9.154725
6,California Bakery,Bakery,45.477951,9.156397
7,Oren,Asian Restaurant,45.474612,9.151339
8,Venchi,Chocolate Shop,45.477766,9.156335
9,Hotel Campion,Hotel Bar,45.474777,9.151247


In [96]:
Categories_1 = Location_1_venues.groupby('categories').count().sort_values(by=['name'], ascending=False)
print(Categories_1)

                      name  lat  lng
categories                          
Italian Restaurant       5    5    5
Plaza                    4    4    4
Sushi Restaurant         3    3    3
Supermarket              3    3    3
Japanese Restaurant      2    2    2
Park                     2    2    2
Cocktail Bar             2    2    2
Asian Restaurant         2    2    2
Steakhouse               1    1    1
Shopping Plaza           1    1    1
Sandwich Place           1    1    1
Salad Place              1    1    1
Other Great Outdoors     1    1    1
Multiplex                1    1    1
Mobile Phone Shop        1    1    1
Mexican Restaurant       1    1    1
American Restaurant      1    1    1
Hotel Bar                1    1    1
Hotel                    1    1    1
Gym                      1    1    1
Department Store         1    1    1
Chocolate Shop           1    1    1
Campanian Restaurant     1    1    1
Café                     1    1    1
Burger Joint             1    1    1
B

In [25]:
print('There are {} unique categories.'.format(len(Location_1_venues['categories'].unique())))

There are 29 unique categories.


### I find the coordinates of the second possible location

In [31]:
address = 'Piazza Sempione, Milano, Italia'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x2 = location.latitude
longitude_y2 = location.longitude
print('The geographical coordinates of Piazza Sempione are {}, {}.'.format(latitude_x1,longitude_y1))



The geographical coordinates of Piazza Sempione are 45.4745032, 9.15559394876567.


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


### I create the map of Location_2

In [32]:
map_location_2 = folium.Map(location=[latitude_x2,longitude_y2], zoom_start=20)
map_location_2

### I look for venues in the second Location

In [34]:
lat_2 = latitude_x2
lon_2 = longitude_y2
nei_name = 'Location_2'
print('Latitude and longitude of {} are {} and {}'.format(nei_name, lat_2, lon_2))

Latitude and longitude of Location_2 are 45.4759963 and 9.1716504


In [37]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat_2, 
    lon_2, 
    radius, 
    LIMIT)

url




'https://api.foursquare.com/v2/venues/explore?&client_id=HMN0ROLECZHKTAGFQS4KCTMWDWX5FPCWEMWM5AABQNRAV5LN&client_secret=BYA5TUW2DOC02Z5NZZ5RLUPOAGWGKFQCLL4CFWMEI2VZIKUB&v=20180609&ll=45.4759963,9.1716504&radius=500&limit=100'

In [38]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c5424559fb6b767cddd4176'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Parco Sempione',
  'headerFullLocation': 'Parco Sempione, Milan',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 61,
  'suggestedBounds': {'ne': {'lat': 45.4804963045, 'lng': 9.178055920735451},
   'sw': {'lat': 45.471496295499996, 'lng': 9.16524487926455}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b05887af964a5205bc822e3',
       'name': 'Arco della Pace',
       'location': {'address': 'Piazza Sempione',
        'lat': 45.47572835143987,
        'lng': 9.172446727752686,
        'labeledLatLngs': [{'label': 'display',
          'lat': 45.47572

In [40]:
venues = results['response']['groups'][0]['items']
    
Location_2_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Location_2_venues =Location_2_venues.loc[:, filtered_columns]


Location_2_venues['venue.categories'] = Location_2_venues.apply(get_category_type, axis=1)


Location_2_venues.columns = [col.split(".")[-1] for col in Location_2_venues.columns]

Location_2_venues.head(10)


,name,categories,lat,lng
0,Arco della Pace,Monument / Landmark,45.475728,9.172447
1,Tara,Indian Restaurant,45.477730,9.172740
2,Bottega Ghiotta Gourmet,Mediterranean Restaurant,45.476990,9.170726
3,Starita,Pizza Place,45.476821,9.170185
4,Piazza Sempione,Plaza,45.476154,9.172181
5,Maido!,Japanese Restaurant,45.476783,9.172668
6,Bomaki,Sushi Restaurant,45.477299,9.170381
7,Parco Sempione,Park,45.474502,9.176183
8,adidas Runbase,Gym,45.477511,9.170063
9,Cavalli Club Milano,Nightclub,45.473403,9.173201


In [41]:
Location_2_venues.shape

(61, 4)

In [94]:
Categories_2 = Location_2_venues.groupby('categories').count().sort_values(by=['name'], ascending=False)
print(Categories_2)

                              name  lat  lng
categories                                  
Italian Restaurant               8    8    8
Cocktail Bar                     6    6    6
Japanese Restaurant              3    3    3
Pizza Place                      3    3    3
Ice Cream Shop                   2    2    2
Sandwich Place                   2    2    2
Nightclub                        2    2    2
Gastropub                        2    2    2
Sushi Restaurant                 2    2    2
Café                             2    2    2
Bookstore                        2    2    2
Bakery                           2    2    2
Sculpture Garden                 1    1    1
Scenic Lookout                   1    1    1
Theater                          1    1    1
Pub                              1    1    1
Plaza                            1    1    1
Theme Park Ride / Attraction     1    1    1
Playground                       1    1    1
Park                             1    1    1
Outlet Sto

In [64]:
print('There are {} unique categories.'.format(len(Location_2_venues['categories'].unique())))

There are 37 unique categories.


### I find the  coordinates of the third possible location

In [52]:
address = 'Via Cesare Balbo, Milano, Italia'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x3 = location.latitude
longitude_y3 = location.longitude
print('The geographical coordinates of Via Cesare Balbo are {}, {}.'.format(latitude_x3,longitude_y3))



/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geographical coordinates of Via Cesare Balbo are 45.447565, 9.1959873.


### I create the map of Location_3

In [53]:
map_location_3 = folium.Map(location=[latitude_x3,longitude_y3], zoom_start=20)
map_location_3

### I look for venues in the third location

In [54]:
lat_3 = latitude_x3
lon_3 = longitude_y3
nei_name = 'Location_3'
print('Latitude and longitude of {} are {} and {}'.format(nei_name, lat_3, lon_3))

Latitude and longitude of Location_3 are 45.447565 and 9.1959873


In [58]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat_3, 
    lon_3, 
    radius, 
    LIMIT)

url





'https://api.foursquare.com/v2/venues/explore?&client_id=HMN0ROLECZHKTAGFQS4KCTMWDWX5FPCWEMWM5AABQNRAV5LN&client_secret=BYA5TUW2DOC02Z5NZZ5RLUPOAGWGKFQCLL4CFWMEI2VZIKUB&v=20180609&ll=45.447565,9.1959873&radius=500&limit=100'

In [59]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c5429284434b9740e57ca29'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Zona 5',
  'headerFullLocation': 'Zona 5, Milan',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 61,
  'suggestedBounds': {'ne': {'lat': 45.4520650045, 'lng': 9.202389591350462},
   'sw': {'lat': 45.443064995499995, 'lng': 9.189585008649539}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '540df6ac498e9d0f4506d283',
       'name': 'Zinc Bar',
       'location': {'address': 'via Balbo, 36',
        'lat': 45.44654731635615,
        'lng': 9.196083430232425,
        'labeledLatLngs': [{'label': 'display',
          'lat': 45.44654731635615,
          'ln

In [60]:
venues = results['response']['groups'][0]['items']
    
Location_3_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Location_3_venues =Location_3_venues.loc[:, filtered_columns]


Location_3_venues['venue.categories'] = Location_3_venues.apply(get_category_type, axis=1)


Location_3_venues.columns = [col.split(".")[-1] for col in Location_3_venues.columns]

Location_3_venues.head(10)

,name,categories,lat,lng
0,Zinc Bar,Cocktail Bar,45.446547,9.196083
1,Trattoria Dal Pescatore,Seafood Restaurant,45.449430,9.197994
2,J's Hiro,Japanese Restaurant,45.449453,9.194259
3,Circolo Arci Bellezza,Bar,45.448931,9.194944
4,Mamajuana Restaurante,Caribbean Restaurant,45.448128,9.197559
5,La Cantinetta,Italian Restaurant,45.448304,9.197649
6,Mercato di Campagna Amica,Farmers Market,45.445891,9.199094
7,Caffe Ristretto - da Gianni,Bar,45.446340,9.195253
8,Piadineria,Piadineria,45.448104,9.197538
9,Potafiori,Bistro,45.450401,9.194595


In [61]:
Location_3_venues.shape

(61, 4)

In [93]:
Categories_3 = Location_3_venues.groupby('categories').count().sort_values(by=['name'], ascending=False)
print(Categories_3)

                            name  lat  lng
categories                                
Café                           6    6    6
Pizza Place                    6    6    6
Cocktail Bar                   5    5    5
Wine Bar                       4    4    4
Sushi Restaurant               3    3    3
Bistro                         3    3    3
Restaurant                     3    3    3
Seafood Restaurant             2    2    2
Pub                            2    2    2
Piadineria                     2    2    2
Italian Restaurant             2    2    2
Bar                            2    2    2
Diner                          1    1    1
Farmers Market                 1    1    1
Theater                        1    1    1
Supermarket                    1    1    1
Boarding House                 1    1    1
Sandwich Place                 1    1    1
Burger Joint                   1    1    1
Candy Store                    1    1    1
Pool                           1    1    1
Caribbean R

In [65]:
print('There are {} unique categories.'.format(len(Location_3_venues['categories'].unique())))

There are 33 unique categories.


 ### I find the coordinates of the fourth possible location

In [69]:
address = 'Via Paolo Giovio, Milano, Italia'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude_x4 = location.latitude
longitude_y4 = location.longitude
print('The geographical coordinates of Via Paolo Giovio are {}, {}.'.format(latitude_x4,longitude_y4))




/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geographical coordinates of Via Paolo Giovio are 45.462245, 9.1622524.


### I create the map of location 4

In [70]:
map_location_4 = folium.Map(location=[latitude_x4,longitude_y4], zoom_start=20)
map_location_4

### I look for venues in the fourth location

In [72]:
lat_4 = latitude_x4
lon_4 = longitude_y4
nei_name = 'Location_4'
print('Latitude and longitude of {} are {} and {}'.format(nei_name, lat_4, lon_4))

Latitude and longitude of Location_4 are 45.462245 and 9.1622524


In [73]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat_4, 
    lon_4, 
    radius, 
    LIMIT)

url




'https://api.foursquare.com/v2/venues/explore?&client_id=HMN0ROLECZHKTAGFQS4KCTMWDWX5FPCWEMWM5AABQNRAV5LN&client_secret=BYA5TUW2DOC02Z5NZZ5RLUPOAGWGKFQCLL4CFWMEI2VZIKUB&v=20180609&ll=45.462245,9.1622524&radius=500&limit=100'

In [74]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c542de9f594df20f26e2241'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Washington',
  'headerFullLocation': 'Washington, Milan',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 33,
  'suggestedBounds': {'ne': {'lat': 45.466745004500005,
    'lng': 9.168656358182808},
   'sw': {'lat': 45.4577449955, 'lng': 9.155848441817191}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd7d74de914a593521b52fa',
       'name': 'Bottiglieria Bulloni',
       'location': {'address': 'Via Lipari 2',
        'crossStreet': 'Via Paolo Giovio',
        'lat': 45.46154482100329,
        'lng': 9.16274884824616,
        'labeledLatLngs': [{'lab

In [75]:
venues = results['response']['groups'][0]['items']
    
Location_4_venues = json_normalize(venues) 


filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Location_4_venues =Location_4_venues.loc[:, filtered_columns]


Location_4_venues['venue.categories'] = Location_4_venues.apply(get_category_type, axis=1)


Location_4_venues.columns = [col.split(".")[-1] for col in Location_4_venues.columns]

Location_4_venues.head(10)

,name,categories,lat,lng
0,Bottiglieria Bulloni,Wine Bar,45.461545,9.162749
1,Antica Trattoria Salutati,Italian Restaurant,45.463291,9.159342
2,Bianca,Italian Restaurant,45.464779,9.163074
3,Spazio Rossana Orlandi,Design Studio,45.463812,9.166722
4,Hana Restaurant,Japanese Restaurant,45.466194,9.159510
5,Acqua Pazza,Italian Restaurant,45.465379,9.165187
6,Amedeo D.,Clothing Store,45.465886,9.161631
7,GetFIT,Gym,45.462470,9.167585
8,Piazzale Aquileia,Plaza,45.460792,9.163271
9,Bar del Cantun,Café,45.464244,9.160850


In [91]:
Categories_4 = Location_4_venues.groupby('categories').count().sort_values(by=['name'], ascending=False)
print(Categories_4)

                      name  lat  lng
categories                          
Italian Restaurant       6    6    6
Ice Cream Shop           3    3    3
Chinese Restaurant       2    2    2
Plaza                    2    2    2
Pizza Place              2    2    2
Japanese Restaurant      2    2    2
Bistro                   1    1    1
Theater                  1    1    1
Sandwich Place           1    1    1
Movie Theater            1    1    1
Juice Bar                1    1    1
Gym / Fitness Center     1    1    1
Breakfast Spot           1    1    1
Gym                      1    1    1
Food Truck               1    1    1
Design Studio            1    1    1
Department Store         1    1    1
Cocktail Bar             1    1    1
Clothing Store           1    1    1
Cheese Shop              1    1    1
Café                     1    1    1
Wine Bar                 1    1    1


In [79]:
print('There are {} unique categories.'.format(len(Location_4_venues['categories'].unique())))

There are 22 unique categories.


In [97]:
Categories_1.to_csv('Categories_1.csv')

In [98]:
Categories_2.to_csv('Categories_2.csv')

In [99]:
Categories_3.to_csv('Categories_3.csv')

In [100]:
Categories_4.to_csv('Categories_4.csv')